In [2]:
import ee
import geemap
from datetime import datetime
ee.Authenticate()
ee.Initialize()

In [3]:
# define an area of interest
# roi = ee.Geometry.Point([-118.193264, 34.124524]);
roi = ee.Geometry.Rectangle([-119, 33.638, -117.607, 34.403])

In [4]:
# get MODIS land surface temperature dataset
d1 = '2022-07-01'
d2 = '2022-09-04'
dataset = ee.ImageCollection('MODIS/061/MOD11A1').filter(ee.Filter.date(d1, d2))
lst_k = dataset.select('LST_Day_1km')

In [5]:
# define function to convert modis temps (scaled by 0.02, kelvin) to fahrenheit and clip to roi
def convert_to_f(lst_modis):
    lst_c = lst_modis.multiply(0.02).add(-273.15)
    lst_f = lst_c.multiply(1.8).add(32)
    lst_f = lst_f.clip(roi)
    return lst_f

In [6]:
# convert modis temps to fahrenheit
lst_f = lst_k.map(convert_to_f)

In [7]:
# get dates of each image
n = dataset.size().getInfo()
d = []
for i in range(0,n):
    img = ee.Image(dataset.toList(n).get(i))
    dt = img.get('system:index').getInfo()
    dt = datetime.strptime(str(dt), "%Y_%m_%d").strftime("%Y-%b-%d")
    d.append(dt)

In [35]:
# define visualization parameters
lst_f_vis = {
  'min': 85.0,
  'max': 120.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003', '630a01',
  ],
  'opacity': 0.5,
};

# show any image on the map
idx = 0
img = ee.Image(lst_f.toList(n).get(idx))

Map = geemap.Map()
#Map.centerObject(roi, 8)
Map.setCenter(-118.3, 34.05448, 10)
Map.addLayer(img, lst_f_vis, d[idx])
Map

Map(center=[34.05448, -118.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

60
60
2022-Jul-01
